In [9]:
import pandas as pd
import numpy as np

In [10]:
df=pd.read_csv('data/labeled_reviews.csv')
df.head()

,Movie_N,cleaned_review,Stars,label
0,The Old Guard,love concept execution not bad there some crin...,7,1
1,The Old Guard,there were just so many clichés cringe naive m...,5,1
2,The Old Guard,actors all good well thats story childlike dir...,5,1
3,The Old Guard,my mind kept referring back austin powers badd...,9,2
4,The Old Guard,london scenes end were just bizarre empty stre...,5,1


In [11]:
df.dropna(subset=['cleaned_review'], inplace=True)


In [12]:
df['Stars']=df['Stars'].astype(int)

In [13]:
df

,Movie_N,cleaned_review,Stars,label
0,The Old Guard,love concept execution not bad there some crin...,7,1
1,The Old Guard,there were just so many clichés cringe naive m...,5,1
2,The Old Guard,actors all good well thats story childlike dir...,5,1
3,The Old Guard,my mind kept referring back austin powers badd...,9,2
4,The Old Guard,london scenes end were just bizarre empty stre...,5,1
...,...,...,...,...
5676,Greyhound,my only complaint movie could have been longer...,5,1
5677,Greyhound,though movie might have been right technical t...,8,1
5678,Greyhound,i felt movie could have been longer better dep...,1,0
5679,Greyhound,climax if any not up mark,9,2


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import joblib
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier  # BEGIN:

from sklearn.svm import LinearSVC  # END:

In [15]:
X_text = df["cleaned_review"]
y = df["label"]

In [16]:
vector=joblib.load("models/tfidf_vectorizer.pkl")
X=vector.fit_transform(X_text)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [500, 1000]
}
grid = GridSearchCV(LogisticRegression(multi_class="multinomial"), param_grid, cv=3, scoring="accuracy")
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
model = grid.best_estimator_


c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, 

Best parameters: {'C': 1, 'max_iter': 1000, 'solver': 'saga'}


In [24]:

model.score(X_test, y_test)

0.5035310734463276

In [25]:
arr=model.predict(X_test)
arr

array([2, 1, 0, ..., 2, 2, 0])

In [30]:
review="it was great and i loved it"
def predict_sentiment(review):
    review_vectorized = vector.transform([review])
    prediction = model.predict(review_vectorized)
    x=prediction[0]
    if x==1:
        print("Neutral")
    elif x==0:
        print("Negative")
    else:
        print("Positive")
predict_sentiment(review)

Positive


In [31]:
joblib.dump(model, "models/sentiment_model.pkl")
print("Model saved as sentiment_model.pkl")

Model saved as sentiment_model.pkl
